# Business Informatics
# 2nd assignment: fishing lake problem (linear programming)
#### submitted by: Maximilian3141
#### Date: 2023.11.16
\
The goal is to maximize the entertainment value of fishing at the lake while adhering to budget constraints set by three different companies (A, B, and C).

### Decision Variables
Let $x_c$ ,$x_b$ and $x_p$ represent the amount of carp, bream, and perch caught, respectively.

### Objective Function
Maximize the entertainment value ( $E$ ):

$ E = 3x_c + 2x_b + 4x_p $

### Constraints
1. Company A's budget constraint:

$ x_c + x_b + 2x_p \leq 4 $

2. Company B's budget constraint:

$ 2x_c + 3x_b \leq 5 $

3. Company C's budget constraint:

$ 2x_c + 2x_b + 4x_p \leq 7 $

4. Non-negativity constraints:

$ x_c, x_b, x_p \geq 0 $

### Final Linear Programming Model
Maximize:

$$ E = 3x_c + 2x_b + 4x_p $$

Subject to:

\begin{align*}
x_c + x_b + 2x_p &\leq 4 \\
2x_c + 3x_b &\leq 5 \\
2x_c + 2x_b + 4x_p &\leq 7 \\
x_c, x_b, x_p &\geq 0
\end{align*}

This model aims to optimize the entertainment of fishing while respecting the budget constraints of the three companies.


In [5]:
# Install dependencies
%pip install -q amplpy

In [6]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


### CBC Solver

In [21]:
# Create an AMPL instance
ampl = AMPL()

# Load the AMPL model from a string (the model provided earlier)
ampl.eval("""
set FishTypes;
set Companies;

param EntertainmentValue{FishTypes};
param Price{Companies, FishTypes};
param MaxBudget{Companies};

var Catch{FishTypes} >= 0;

maximize TotalEntertainment:
    sum{f in FishTypes} EntertainmentValue[f] * Catch[f];

subject to CompanyBudgetConstraint{c in Companies}:
    sum{f in FishTypes} Price[c, f] * Catch[f] <= MaxBudget[c];
""")

# Set the data for the model
fish_types = ampl.getSet("FishTypes")
companies = ampl.getSet("Companies")

entertainment_values = {'carp': 3, 'bream': 2, 'perch': 4}
prices = {'A': {'carp': 1, 'bream': 1, 'perch': 2},
          'B': {'carp': 2, 'bream': 3, 'perch': 0},
          'C': {'carp': 2, 'bream': 2, 'perch': 4}}
max_budgets = {'A': 4, 'B': 5, 'C': 7}

fish_types.setValues(entertainment_values.keys())
companies.setValues(prices.keys())

for fish_type in fish_types:
    ampl.getParameter('EntertainmentValue').setValues({fish_type: entertainment_values[fish_type]})
    for company in companies:
        ampl.getParameter('Price').setValues({(company, fish_type): prices[company][fish_type]})
    ampl.getParameter('MaxBudget').setValues({company: max_budgets[company] for company in companies})

# Specify the solver
ampl.setOption('solver', 'cbc')  # cbc or ipopt

# Solve the optimization problem
ampl.solve()

# Display the results
print("Optimal Catch:")
for fish_type in fish_types:
    print(f"{fish_type}: {ampl.getVariable('Catch')[fish_type].value()} kilograms")

total_entertainment = ampl.getObjective("TotalEntertainment").value()
print(f"Total Entertainment Value: {total_entertainment:.2f}")

# Close the AMPL instance
ampl.close()

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 9.5
0 simplex iterations
 
Optimal Catch:
carp: 2.5 kilograms
bream: 0.0 kilograms
perch: 0.5 kilograms
Total Entertainment Value: 9.50


### IPOPT Solver

In [22]:
# Create an AMPL instance
ampl = AMPL()

# Load the AMPL model from a string (the model provided earlier)
ampl.eval("""
set FishTypes;
set Companies;

param EntertainmentValue{FishTypes};
param Price{Companies, FishTypes};
param MaxBudget{Companies};

var Catch{FishTypes} >= 0;

maximize TotalEntertainment:
    sum{f in FishTypes} EntertainmentValue[f] * Catch[f];

subject to CompanyBudgetConstraint{c in Companies}:
    sum{f in FishTypes} Price[c, f] * Catch[f] <= MaxBudget[c];
""")

# Set the data for the model
fish_types = ampl.getSet("FishTypes")
companies = ampl.getSet("Companies")

entertainment_values = {'carp': 3, 'bream': 2, 'perch': 4}
prices = {'A': {'carp': 1, 'bream': 1, 'perch': 2},
          'B': {'carp': 2, 'bream': 3, 'perch': 0},
          'C': {'carp': 2, 'bream': 2, 'perch': 4}}
max_budgets = {'A': 4, 'B': 5, 'C': 7}

fish_types.setValues(entertainment_values.keys())
companies.setValues(prices.keys())

for fish_type in fish_types:
    ampl.getParameter('EntertainmentValue').setValues({fish_type: entertainment_values[fish_type]})
    for company in companies:
        ampl.getParameter('Price').setValues({(company, fish_type): prices[company][fish_type]})
    ampl.getParameter('MaxBudget').setValues({company: max_budgets[company] for company in companies})

# Specify the solver
ampl.setOption('solver', 'ipopt')  # cbc or ipopt

# Solve the optimization problem
ampl.solve()

# Display the results
print("Optimal Catch:")
for fish_type in fish_types:
    print(f"{fish_type}: {ampl.getVariable('Catch')[fish_type].value()} kilograms")

total_entertainment = ampl.getObjective("TotalEntertainment").value()
print(f"Total Entertainment Value: {total_entertainment:.2f}")

# Close the AMPL instance
ampl.close()

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot